In [81]:
import numpy as np
import pandas as pd
from random import random
from math import exp
from random import seed

In [82]:
data = pd.read_csv('data/seeds_dataset.txt',header=None,sep=' ')

In [83]:
data.head(5)

,0
0,15.26\t14.84\t0.871\t5.763\t3.312\t2.221\t5.22\t1
1,14.88\t14.57\t0.8811\t5.554\t3.333\t1.018\t4.9...
2,14.29\t14.09\t0.905\t5.291\t3.337\t2.699\t4.82...
3,13.84\t13.94\t0.8955\t5.324\t3.379\t2.259\t4.8...
4,16.14\t14.99\t0.9034\t5.658\t3.562\t1.355\t5.1...


# Initialize the network

In [84]:
def initialize_network(n_inputs, n_hidden,n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [85]:
#test network
#network = initialize_network(2,1,2)
#for layer in network:
#    print layer

# Activation of neurons

In [86]:
def activate(weights,inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return activation

# Transfer neuron activation

In [87]:
def transfer(activation):
    return 1.0/(1.0 + math.exp(-activation))

# Forward propagation

In [88]:
def forward_propagate(network,row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'],inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [89]:
# Testing forward propagation
#network = initialize_network(2,1,2)
#row = [1,0,None]     // Last one is the dummy input(bias 1)
#output = forward_propagate(network,row)

# Back propagation

In [90]:
def transfer_derivative(output):
    return output*(1-output)

In [91]:
def back_propagate_error(network,expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error += (neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j]-neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j]*transfer_derivative(neuron['output'])

In [92]:
# Testing backpropagation
#expected = [0,1]
#back_propagate_error(network,expected)

In [93]:
#for layer in network:
#    print layer

# Updating network weights after getting errors

In [94]:
def update_weights(network,row,l_rate): #l_rate:learining rate
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1] += l_rate*neuron['delta']

# Training network

In [95]:
def train_network(network,train,l_rate,n_epoch,n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network,row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            back_propagate_error(network,expected)
            update_weights(network,row,l_rate)
        print "Iteration : ",epoch,", Learning rate : ",l_rate,", Sum of errors : ",sum_error
    

In [96]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0])-1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs,2,n_outputs)
train_network(network,dataset,0.5,20,n_outputs)


Iteration :  0 , Learning rate :  0.5 , Sum of errors :  6.35012662902
Iteration :  1 , Learning rate :  0.5 , Sum of errors :  5.53148430656
Iteration :  2 , Learning rate :  0.5 , Sum of errors :  5.22116507016
Iteration :  3 , Learning rate :  0.5 , Sum of errors :  4.9513841342
Iteration :  4 , Learning rate :  0.5 , Sum of errors :  4.51894354301
Iteration :  5 , Learning rate :  0.5 , Sum of errors :  4.17304061768
Iteration :  6 , Learning rate :  0.5 , Sum of errors :  3.83493643241
Iteration :  7 , Learning rate :  0.5 , Sum of errors :  3.5057278939
Iteration :  8 , Learning rate :  0.5 , Sum of errors :  3.19211953585
Iteration :  9 , Learning rate :  0.5 , Sum of errors :  2.89766994075
Iteration :  10 , Learning rate :  0.5 , Sum of errors :  2.62554309287
Iteration :  11 , Learning rate :  0.5 , Sum of errors :  2.3774366973
Iteration :  12 , Learning rate :  0.5 , Sum of errors :  2.15347622324
Iteration :  13 , Learning rate :  0.5 , Sum of errors :  1.95270167724
Itera

In [98]:
for layer in network:
    print layer
    print ""

[{'output': 0.029980305604426185, 'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'delta': -0.0059546604162323625}, {'output': 0.9456229000211323, 'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'delta': 0.0026279652850863837}]

[{'output': 0.23648794202357587, 'weights': [2.515394649397849, -0.3391927502445985, -0.9671565426390275], 'delta': -0.04270059278364587}, {'output': 0.7790535202438367, 'weights': [-2.5584149848484263, 1.0036422106209202, 0.42383086467582715], 'delta': 0.03803132596437354}]



# prediction

In [101]:
def predict(network,row):
    outputs = forward_propagate(network,row)
    return outputs.index(max(outputs))

In [102]:
for row in dataset:
    prediction = predict(network,row)
    print "Expected : ",row[-1], ", Got : ",prediction

Expected :  0 , Got :  0
Expected :  0 , Got :  0
Expected :  0 , Got :  0
Expected :  0 , Got :  0
Expected :  0 , Got :  0
Expected :  1 , Got :  1
Expected :  1 , Got :  1
Expected :  1 , Got :  1
Expected :  1 , Got :  1
Expected :  1 , Got :  1


# Binding things together

In [ ]:
def back_propagation(train,test,l_rate,n_epoch,n_hidden):
    n_inputs = len(train[0])-1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs,n_hidden,n_outputs)
    train_network(network,train,l_rate,n_epoch,n_outputs)
    prediction = list()
    for row in test:
        prediction = predict(network,row)
        prediction.append(prediction)
    return prediction
